<a href="https://colab.research.google.com/github/Ritukuklani/Kaggle-Virality-Predictor/blob/main/viralitypredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I considered the number of likes, views, comments, bookmarks and number of follows to generate a virality score for each contentID.

# I observed that the following was a regression problem ans hence used linear regression and gradient descent to the problem.

# I used the R2 score and the Mean Square Error evaluation metrics.

# If I had more time I would like to check out the accuracy provided my Stochastic Gradient Descent as that would be a faster approach.

# Moreover, I would like to use some unsupervised learning techniques before applying gradient descent or linear regression and check the accuracy obtained.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pwd

/content


#Loading the dataframes

In [ ]:
df_shared_articles = pd.read_csv('sample_data/shared_articles.csv')
df_shared_articles.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [ ]:
df_user_interactions = pd.read_csv('sample_data/users_interactions.csv')
df_user_interactions

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR
72308,1485190425,VIEW,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR
72309,1485190072,VIEW,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR
72310,1485190434,VIEW,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR


#Getting a count of the events

In [ ]:
df_user_interactions['COUNTER'] =1  
data = df_user_interactions.groupby(['contentId','eventType'])['COUNTER'].sum().reset_index() 
data.head(25)

,contentId,eventType,COUNTER
0,-9222795471790223670,BOOKMARK,1
1,-9222795471790223670,COMMENT CREATED,2
2,-9222795471790223670,FOLLOW,3
3,-9222795471790223670,LIKE,4
4,-9222795471790223670,VIEW,16
5,-9216926795620865886,BOOKMARK,1
6,-9216926795620865886,COMMENT CREATED,1
7,-9216926795620865886,FOLLOW,1
8,-9216926795620865886,LIKE,3
9,-9216926795620865886,VIEW,15


#Creating a pivot table using the contentId as the row and the eventTypes as the columns and filling the missing values with 0.

In [ ]:
final_df = data.pivot_table('COUNTER', ['contentId'], 'eventType')
final_df = final_df.fillna(0)

In [ ]:
final_df


eventType,BOOKMARK,COMMENT CREATED,FOLLOW,LIKE,VIEW
contentId,,,,,
-9222795471790223670,1.0,2.0,3.0,4.0,16.0
-9216926795620865886,1.0,1.0,1.0,3.0,15.0
-9194572880052200111,2.0,1.0,1.0,4.0,21.0
-9192549002213406534,0.0,1.0,0.0,5.0,50.0
-9190737901804729417,0.0,0.0,0.0,1.0,8.0
...,...,...,...,...,...
9213260650272029784,0.0,0.0,0.0,0.0,11.0
9215261273565326920,3.0,0.0,0.0,3.0,24.0
9217155070834564627,0.0,2.0,0.0,0.0,14.0


In [ ]:
print(final_df.shape)

(2987, 5)


#Creating virality label for the dataset

In [ ]:
def create_label(row):
   return (1* row['VIEW']) + (4*row['LIKE']) + (10*row['COMMENT CREATED']) +( 25*row['FOLLOW'] )+ (100*row['BOOKMARK'])

final_df['label'] = final_df.apply (lambda row: create_label(row), axis=1)

In [ ]:
x = final_df.iloc[0:, 0:5]
y = final_df.iloc[0:, 5:]
y = np.squeeze(y)
y.shape
type(y)

pandas.core.series.Series

In [ ]:
final_df.describe()       #We can observe the virality scores for the dataset.

eventType,BOOKMARK,COMMENT CREATED,FOLLOW,LIKE,VIEW,label
count,2987.000000,2987.000000,2987.000000,2987.000000,2987.000000,2987.000000
mean,0.824573,0.539337,0.471041,1.923334,20.450619,127.770673
std,2.231426,1.292221,1.076175,3.368695,27.199921,259.044552
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,5.000000,8.000000
50%,0.000000,0.000000,0.000000,1.000000,12.000000,43.000000
75%,1.000000,1.000000,1.000000,2.000000,25.000000,150.000000
max,78.000000,13.000000,14.000000,53.000000,355.000000,7907.000000


#From the dataset shown above, we see that this is a regression problem. To solve this, we could linear regression model.

#Method 1:

#**Linear Regression**

We have split the dataset into training and testing data (80% and 20% respectively)

We can observe that the Mean Squared Loss for the training data set is 6.02*10^-31

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(final_df, test_size=0.2)
X_train, y_train = train.drop('label',axis = 1), train['label']
X_test, y_test = test.drop('label',axis = 1), test['label']

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
model = LinearRegression().fit(X_train, y_train)


#We can observe that the weights trained out are [100(bookmark), 10(comment), 25(follow), 4(like), 1(view)]

In [ ]:
print('intercept:', model.intercept_)
print('Weights trained', model.coef_)  #The weights trained by the model.

intercept: -3.126388037344441e-13
Weights trained [100.  10.  25.   4.   1.]


#The mean squared error for the test data comes out to be 8.13*10^-26

In [ ]:
y_pred2 = model.predict(X_test)
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred2)

2.7776593470058668e-25

#Method 2:

#**Gradient Descent**

#We now try to apply gradient descent which is much more optimum method to solve regression problems.

In [ ]:
data = pd.DataFrame(x).to_numpy()
target = pd.DataFrame(y).to_numpy()

X = data
y = target
X.shape
y = np.squeeze(y)

#The input data is normalized to get better accuracy while applying the gradient descent algorithm.

In [ ]:
X = (X - X.mean())/X.std()
y = (y - y.mean())/y.std()

In [ ]:
X

array([[-0.26327869, -0.19474832, -0.12621794, -0.05768757,  0.76467692],
       [-0.26327869, -0.26327869, -0.26327869, -0.12621794,  0.69614655],
       [-0.19474832, -0.26327869, -0.26327869, -0.05768757,  1.10732879],
       ...,
       [-0.33180907, -0.19474832, -0.33180907, -0.33180907,  0.62761617],
       [-0.33180907, -0.33180907, -0.33180907, -0.19474832,  3.09470965],
       [-0.33180907, -0.33180907, -0.33180907, -0.33180907,  0.0108428 ]])

In [ ]:
train_pct_index = int(0.8 * len(X))
X_train, X_test = X[:train_pct_index], X[train_pct_index:]
y_train, y_test = y[:train_pct_index], y[train_pct_index:]

In [ ]:
X_test.shape

(598, 5)

#Initialize the weights

In [ ]:
def initialize_w(lenw):
  w= np.zeros(lenw)
  return w

#Initialize the loss function

In [ ]:
def loss(x,y,w):
  n=len(x)
  #for i in range(n):
  dw= np.dot(x.T,(y-np.dot(x,w)))
  return dw

#The gradient descent algorithm

In [ ]:
def gradient_descent(X, y, l_rate, epochs):
  m=len(y)
  lenw= X.shape[1]
  w=initialize_w(lenw)
  loss_history=[0]* epochs
  for i in range ( epochs):
    g= X.T.dot(X.dot(w)-y)/m      
    w= w-l_rate*g  #Updating the value of the weight function at every epoch
    loss_value= loss(X,y,w)
    loss_history[i]= loss_value
  return w, loss_history

#Calculate the R2 score or the prediction accuracy for the model

In [ ]:
def R2_score(y,y_pred): 
  y_mean= np.mean(y)
  r2_d=sum((y-y_mean)**2)
  r2_n=sum((y-y_pred)**2)
  r2_val=1-(r2_n/r2_d)
  return r2_val

#An accuracy of 87.35% is achieved for the training data set.
#We can observe that the weights are scaled down to a different value as compared to [100(bookmark), 10(comment), 25(follow), 4(like), 1(view)] because we normalized the input data and also because the value of the weights depends on the initialization of the weights. However, we get a accuracy of 87.35% for the model by applying gradient descent for the training dataset.

In [ ]:
l_rate=0.01
epochs=10000

w, loss_history = gradient_descent(X_train,y_train,l_rate,epochs)
print("Weights obtained after training", w)
y_pred = X_train.dot(w)
r2 = R2_score(y_train,y_pred)
print("R2 score for train data: ",r2)

Weights obtained after training [ 3.98034119e+00 -1.09930107e+00 -1.37416702e+00  2.99207660e-03
  2.71554856e-01]
R2 score for train data:  0.8735158343722751


#An accuracy of 83.13% is achieved for the test data set.

In [ ]:
y_pred = X_test.dot(w)
r2 = R2_score(y_test,y_pred)
print("R2 score for test data:",r2)

R2 score for test data: 0.8313112915615459
